In [2]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
import kagglehub
import os
import shutil

# Download latest version to the default cache location
path = kagglehub.dataset_download("harshpatel1326/zooplankton-model-train-data")

print("Path to dataset files:", path)

# Define the destination directory
destination_dir = "/content/"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move all files and directories from the cache to the destination
for item in os.listdir(path):
    s = os.path.join(path, item)
    d = os.path.join(destination_dir, item)
    if os.path.isdir(s):
        shutil.move(s, d)
    else:
        shutil.move(s, d)

print(f"Files moved to {destination_dir}")

100%|██████████| 777M/777M [00:38<00:00, 21.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/harshpatel1326/zooplankton-model-train-data/versions/1
Files moved to /content/


In [4]:
import os
import random
import shutil

# Define the source directory (training dataset)
train_data_dir = "/content/train"

# Define the destination directory for mixed images
mixed_folder = "/content/mixed_images"

# Create the mixed folder if it doesn't exist
os.makedirs(mixed_folder, exist_ok=True)

# Get the list of class names (subfolders in train_data_dir)
class_names = [d for d in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, d))]

# Number of images to copy from each class
num_images_per_class = 15

# Copy random images from each class folder
for class_name in class_names:
    class_folder = os.path.join(train_data_dir, class_name)
    images = [f for f in os.listdir(class_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))] # Filter for image files

    # Ensure there are enough images in the class folder
    if len(images) < num_images_per_class:
        print(f"Warning: Not enough images ({len(images)}) in {class_name} to copy {num_images_per_class}. Copying all available images.")
        selected_images = images
    else:
        selected_images = random.sample(images, num_images_per_class)

    # Copy selected images to the mixed folder
    for img_file in selected_images:
        src_path = os.path.join(class_folder, img_file)
        dest_path = os.path.join(mixed_folder, img_file)
        shutil.copy(src_path, dest_path)
        # Optionally, you can rename the copied file to include the class name if needed
        # shutil.copy(src_path, os.path.join(mixed_folder, f"{class_name}_{img_file}"))

print(f"Copied {num_images_per_class} random images from each class to '{mixed_folder}'")

Copied 15 random images from each class to '/content/mixed_images'


In [17]:
import os
import shutil
import torch
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# ----------------------------
# Configuration
# ----------------------------
input_folder = "/content/mixed_images"        # Folder with unlabeled/mixed images
output_folder = "/content/sorted_output"      # Where sorted images will be copied
model_path = "/content/resnet50_model.pth"    # Your trained model file
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Must match training normalization
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ----------------------------
# Load Model
# ----------------------------
model = torch.load(model_path, map_location=device, weights_only=False)
model.eval()

# Get class names (from training ImageFolder)
# Adjust path to your training dataset folder
train_data_dir = "/content/train"
class_names = sorted(os.listdir(train_data_dir))  # list of class folders

# ----------------------------
# Helper: Predict class of an image
# ----------------------------
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    return class_names[pred.item()]

# ----------------------------
# Process all images
# ----------------------------
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

correct, total = 0, 0
for root, _, files in os.walk(input_folder):
    for img_file in files:
        if not img_file.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(root, img_file)
        pred_class = predict_image(img_path)

        # Copy image to respective class folder
        dest_dir = os.path.join(output_folder, pred_class)
        os.makedirs(dest_dir, exist_ok=True)
        shutil.copy(img_path, dest_dir)

        # Ground truth = parent folder name
        gt_class = os.path.basename(root)
        if gt_class in class_names:
            total += 1
            if pred_class == gt_class:
                correct += 1


# Print accuracy if labels available
if total > 0:
    print(f"Accuracy: {100 * correct / total:.2f}%")
else:
    print("No ground-truth labels found. Images sorted by predicted class.")


No ground-truth labels found. Images sorted by predicted class.


In [15]:
from google.colab import files
import shutil
import os

# Define the folder to be downloaded
folder_to_download = "/content/sorted_output"

# Create a zip file of the folder
zip_filename = "sorted_output.zip"
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_to_download)

# Download the zip file
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
import os
import shutil

mixed_folder = "/content/sample_data"

# Remove the directory and all its contents
if os.path.exists(mixed_folder):
    shutil.rmtree(mixed_folder)
    print(f"Contents of '{mixed_folder}' deleted.")

# Recreate the directory
os.makedirs(mixed_folder)
print(f"'{mixed_folder}' recreated.")

Contents of '/content/sample_data' deleted.
'/content/sample_data' recreated.


In [21]:
import os
import shutil
import random
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# ----------------------------
# Configuration
# ----------------------------
train_data_dir = "/content/train"             # your training dataset (with class subfolders)
test_data_dir = "/content/test"               # your test dataset (with class subfolders)
mixed_folder = "/content/mixed_images"        # will be created: mixed test images
output_folder = "/content/sorted_output"      # classified output
weights_path = "/content/resnet50_weights.pth" # safer: use state_dict file

num_classes = len(os.listdir(train_data_dir))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Step 1: Create Mixed Folder
# ----------------------------
if os.path.exists(mixed_folder):
    shutil.rmtree(mixed_folder)
os.makedirs(mixed_folder)

# Copy all test images (ignoring folder structure) into mixed folder
for class_name in os.listdir(test_data_dir):
    class_path = os.path.join(test_data_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            src = os.path.join(class_path, img_file)
            dst = os.path.join(mixed_folder, f"{class_name}_{img_file}")
            shutil.copy(src, dst)

print(f"✅ Mixed folder created with {len(os.listdir(mixed_folder))} images.")

# ----------------------------
# Step 2: Load Model
# ----------------------------
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

state_dict = torch.load(weights_path, map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

# Class names from training data
class_names = sorted(os.listdir(train_data_dir))

# ----------------------------
# Transform (must match training)
# ----------------------------
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ----------------------------
# Helper: Predict Class
# ----------------------------
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    return class_names[pred.item()]

# ----------------------------
# Step 3: Classify Mixed Images
# ----------------------------
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

correct, total = 0, 0
for img_file in os.listdir(mixed_folder):
    img_path = os.path.join(mixed_folder, img_file)
    if not img_file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    pred_class = predict_image(img_path)

    # Copy to output folder
    dest_dir = os.path.join(output_folder, pred_class)
    os.makedirs(dest_dir, exist_ok=True)
    shutil.copy(img_path, dest_dir)

    # Ground truth is prefix of filename (added during mixing)
    gt_class = img_file.split("_")[0]
    if gt_class in class_names:
        total += 1
        if pred_class == gt_class:
            correct += 1

# ----------------------------
# Step 4: Print Accuracy
# ----------------------------
if total > 0:
    print(f"✅ Classification complete! Accuracy: {100 * correct / total:.2f}% "
          f"({correct}/{total})")
else:
    print("⚠️ No ground-truth available, only sorted images.")


✅ Mixed folder created with 10000 images.
⚠️ No ground-truth available, only sorted images.


In [24]:
import os
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# ----------------------------
# Configuration
# ----------------------------
train_data_dir = "/content/train"             # training folder (for class names)
test_data_dir = "/content/validation"               # test folder (with subfolders per class)
weights_path = "/content/resnet50_weights.pth" # state_dict file you saved

num_classes = len(os.listdir(train_data_dir))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Load Model
# ----------------------------
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

state_dict = torch.load(weights_path, map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

# ----------------------------
# Data Loader for Test Set
# ----------------------------
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(test_data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ----------------------------
# Accuracy Calculation
# ----------------------------
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"✅ Test Accuracy: {accuracy:.2f}% ({correct}/{total})")


✅ Test Accuracy: 93.79% (9379/10000)
